In [ ]:
# set up
%tensorflow_version 1.x
! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html 

# multilingual set up 
!pip install ftfy regex tqdm gdown
!pip install transformers

# Ninja set up
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [1]:
from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

In [15]:
cd Multi_StyleCLIP/

/Users/esoyeon/Projects/Multi_StyleCLIP


In [7]:
# get language model pretrained weights for multilingual clip
!bash get-weights.sh

--2021-12-01 17:46:13--  https://www.dropbox.com/s/s77xw5308jeljlp/Swedish-500k%20Linear%20Weights.pkl
www.dropbox.com (www.dropbox.com) 해석 중... 162.125.84.18
다음으로 연결 중: www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 301 Moved Permanently
위치: /s/raw/s77xw5308jeljlp/Swedish-500k%20Linear%20Weights.pkl [따라감]
--2021-12-01 17:46:13--  https://www.dropbox.com/s/raw/s77xw5308jeljlp/Swedish-500k%20Linear%20Weights.pkl
www.dropbox.com에 기존 연결 재활용:443.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://uced8160adc1358135d6f7c3e912.dl.dropboxusercontent.com/cd/0/inline/BbBEYw6p3z5a-mvinfyfJD7A06DwD83q601jP-xnhAHft5BMgjgnCOKBFDQ9M65sKsBS2mVajwYKqL4jmpajWQjxxaVhEKF7muO0lqXpZNUYxwOo1OQCu8V_HWY_6OaWCZ6Ykl2RZsaP--UupM4Dz583/file# [따라감]
--2021-12-01 17:46:13--  https://uced8160adc1358135d6f7c3e912.dl.dropboxusercontent.com/cd/0/inline/BbBEYw6p3z5a-mvinfyfJD7A06DwD83q601jP-xnhAHft5BMgjgnCOKBFDQ9M65sKsBS2mVajwYKqL4jmpajWQjxxaVhEKF7muO0lqXpZNUYxwOo1OQCu8V_HWY_

In [19]:
dataset_name='ffhq' 

# input prepare data 
!python GetCode.py --dataset_name $dataset_name --code_type 'w' 
!python GetCode.py --dataset_name $dataset_name --code_type 's' 
!python GetCode.py --dataset_name $dataset_name --code_type 's_mean_std' 

Traceback (most recent call last):
  File "GetCode.py", line 7, in <module>
    from dnnlib import tflib  
  File "/Users/esoyeon/Projects/Multi_StyleCLIP/models/StyleCLIP/global_directions/dnnlib/tflib/__init__.py", line 9, in <module>
    from . import autosummary
  File "/Users/esoyeon/Projects/Multi_StyleCLIP/models/StyleCLIP/global_directions/dnnlib/tflib/autosummary.py", line 32, in <module>
    from . import tfutil
  File "/Users/esoyeon/Projects/Multi_StyleCLIP/models/StyleCLIP/global_directions/dnnlib/tflib/tfutil.py", line 18, in <module>
    import tensorflow.contrib   # requires TensorFlow 1.x!
ModuleNotFoundError: No module named 'tensorflow.contrib'
Traceback (most recent call last):
  File "GetCode.py", line 7, in <module>
    from dnnlib import tflib  
  File "/Users/esoyeon/Projects/Multi_StyleCLIP/models/StyleCLIP/global_directions/dnnlib/tflib/__init__.py", line 9, in <module>
    from . import autosummary
  File "/Users/esoyeon/Projects/Multi_StyleCLIP/models/StyleC

In [20]:
imagenet_templates = [
'{}의 나쁜 사진입니다.',
# '많은 {}명의 사진'
'{}의 조각상',
'{} 보기 힘든 사진입니다.',
'{}의 저해상도 사진입니다.',
'{}의 렌더링입니다.',
'{}의 숫자입니다.',
'{}의 나쁜 사진입니다.',
'{}을(를) 자른 사진입니다.',
'{}의 문신입니다.',
'{}자수.',
'{} 보기 힘든 사진입니다.',
'{}의 밝은 사진입니다.',
'깨끗한 {}의 사진입니다.',
'더러운 {}의 사진',
'{}의 어두운 사진입니다.',
'{}을(를) 그립니다.',
'내 {}의 사진',
'플라스틱 {}',
'멋진 {}의 사진',
'{}의 클로즈업 사진입니다.',
'{}의 흑백 사진입니다.',
'{}을(를) 그린 그림입니다.',
'{}을(를) 그린 그림입니다.',
'{}의 픽셀화된 사진입니다.',
'{}의 조각상입니다.',
'{}의 밝은 사진입니다.',
'{}의 자른 사진입니다.',
'플라스틱 {}' ,
'더러운 {}의 사진입니다.',
'{}의 JPEG 손상 사진입니다.',
'{}의 흐릿한 사진입니다.',
'{}의 사진입니다.',
'{}의 좋은 사진입니다.',
'{}의 렌더링입니다.',
'비디오 게임에서 {}입니다.',
'한 {}의 사진입니다.',
'{}의 낙서입니다.',
'{}의 클로즈업 사진입니다.',
'{}의 사진입니다.',
'종이접기 {}',
'비디오 게임에서 {}',
'{}의 스케치.',
'{}의 낙서입니다.',
'종이접기 {}',
'{}의 저해상도 사진입니다.',
'장난감 {}' ,
'{}을(를) 변형한 것입니다.',
'청렴한 {}의 사진입니다.',
'큰 {}의 사진입니다.',
'{}을(를) 해석한 것입니다.',
'멋진 {}의 사진',
'이상한 {}의 사진',
'{}의 흐릿한 사진입니다.',
'만화 {}' ,
'{} 작품' ,
'{}의 스케치.',
'{}자수.',
'{}의 픽셀화된 사진입니다.',
'{}의 itap.',
'{}의 JPEG 손상 사진입니다.',
'{}의 좋은 사진',
'퍼시 {}' ,
'멋진 {}의 사진',
'작은 {}의 사진입니다.',
'이상한 {}의 사진',
'만화 {}' ,
'{}의 기술.',
'{}을(를) 그립니다.',
'큰 {}의 사진입니다.',
'{}의 흑백 사진입니다.',
'퍼시 {}' ,
'어두운 {} 사진입니다.',
'{}의 잇탭.',
'{}개 중 하나',
'장난감 {}' ,
'내 {}의 잇탭이다.',
'멋진 {} 사진',
'작은 {}의 사진입니다.',
'{}의 문신입니다.'
]

In [21]:
imagenet_templates = [
    'a bad photo of a {}.',
#    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]

In [24]:
os.chdir('/Users/esoyeon/Projects/Multi_StyleCLIP/')

In [30]:
from models.global_directions.manipulate import Manipulator
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device) 

M=Manipulator(dataset_name='ffhq') 
fs3=np.load('data/npy/ffhq/fs3.npy')
np.set_printoptions(suppress=True)

image_path = None
img_index = None

# Choose Image Index
Relevant only when editing generated image

In [31]:
img_index =  1

# Choose Mode (and show input image)
Run for both real and generated image

In [ ]:
# mode='generated image' # ['real image', 'generated image']
mode = 'real image'

if mode == 'real image':
  img_index = 0
  latents = torch.load('/content/restyle-encoder/latents.pt')
  w_plus=latents.cpu().detach().numpy()
  dlatents_loaded=M.W2S(w_plus)

  img_indexs=[img_index]
  dlatent_tmp=[tmp[img_indexs] for tmp in dlatents_loaded]

elif mode == 'generated image':
  img_indexs=[img_index]
  dlatent_tmp=[tmp[img_indexs] for tmp in M.dlatents]
M.num_images=len(img_indexs)

M.alpha=[0]
M.manipulate_layers=[0]
codes,out=M.EditOneC(0,dlatent_tmp) 
original=Image.fromarray(out[0,0]).resize((512,512))
M.manipulate_layers=None
original

# input text description 
For example

| Edit  | Neutral Text | Target Text |
| --- | --- | --- |
| Smile  | face  | smiling face |
| Gender  | female face  | male face |
| Blonde hair | face with hair | face with blonde hair |
| Hi-top fade | face with hair | face with Hi-top fade hair |
| Blue eyes | face with eyes | face with blue eyes |


In [ ]:
neutral='눈을 가진 얼굴' # {type:"string"}
target='파란 눈을 가진 얼굴' # {type:"string"}
classnames=[target,neutral]
dt=GetDt(classnames, model, imagenet_templates)

# modify manipulation strength (alhpa) and disentangle threshold (beta)

- *Manipulation strength* - positive values correspond to moving along the target direction.
- *Disentanglement threshold* - large value means more disentangled edit, just a few channels will be manipulated so only the target attribute will change (for example, grey hair). Small value means less disentangled edit, a large number of channels will be manipulated, related attributes will also change (such as wrinkle, skin color, glasses).

- In the terminal, for every manipulation, the number of channels being manipulated is printed (the number is controlled by the attribute (neutral, target) and the disentanglement threshold).

- For color transformation, usually 10-20 channels is enough. For large structure change (for example, Hi-top fade), usually 100-200 channels are required.
- For an attribute (neutral, target), if you give a low disentanglement threshold, there are just few channels (<20) being manipulated, and usually it is not enough for performing the desired edit.

In [ ]:
beta = 0.1 # {type:"slider", min:0.08, max:0.3, step:0.01}
alpha = 10 # {type:"slider", min:-10, max:10, step:0.1}
M.alpha=[alpha]
boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
codes=M.MSCode(dlatent_tmp,boundary_tmp2)
out=M.GenerateImg(codes)
generated=Image.fromarray(out[0,0])#.resize((512,512))


plt.figure(figsize=(20,7), dpi= 100)
plt.subplot(1,2,1)
plt.imshow(original)
plt.title('original')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(generated)
plt.title('manipulated')
plt.axis('off')

# video

Renders a video interpolating from the base image with provided `beta` to the `target_alpha`. (`target_alpha` can be positive or negative)

In [ ]:
from contextlib import contextmanager
import sys, os
from tqdm.notebook import tqdm

beta =  0.15 #@param {type:"number"}
target_alpha =  4.1#@param {type:"number"}
num_frames = 350 #@param {type:"number"}
frame_rate = 60 #@param {type:"number"}

!rm -rf /content/results
!mkdir /content/results

# Mute GetBoundary()
# https://stackoverflow.com/a/25061573
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

def gen_image(beta, alpha, i):
  M.alpha=[alpha]
  with suppress_stdout():
    boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
  codes=M.MSCode(dlatent_tmp,boundary_tmp2)
  out=M.GenerateImg(codes)
  Image.fromarray(out[0,0]).save(f"/content/results/{i:04d}.png")

alphas = np.linspace(0, target_alpha, num_frames)

print("Generating Frames:")
for i, alpha in tqdm(enumerate(alphas), total=num_frames):
  gen_image(beta, alpha, i)

print("Rendering Video...")
result = os.system(f"ffmpeg -y -r {frame_rate} -i /content/results/%04d.png -c:v libx264 -vf fps={frame_rate} -pix_fmt yuv420p /content/styleclip.mp4")
print("Video saved at styleclip.mp4!")

from google.colab import files
files.download('/content/styleclip.mp4')